# int skills makeup work

Christopher Csiszar

### hyperparameter tuning

I haven't worked with linear regressors in a while, so I'll be practicing on those + a simple RF.

Note the linear regressors below have a built in cross validation function, however I will be returning to basics and tuning the models in a simple way.

In [77]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import dateutil.parser
from sklearn.model_selection import train_test_split

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

In [35]:
! pwd

/Users/chrispaul/Desktop/classes/intskills/kechup


In [36]:
train = pd.read_csv('Train.csv')

/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
train = train[0:23900]

In [38]:
train.tail()

,SalesID,SalePrice,under_20k,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saledate,state,ProductGroup,Enclosure
23895,1222558.0,81000.0,0.0,1019939.0,14287.0,121.0,3.0,2006.0,3395.0,12/22/11 0:00,Virginia,TEX,EROPS w AC
23896,1222569.0,23000.0,0.0,1057622.0,3350.0,121.0,3.0,1000.0,0.0,12/22/11 0:00,Virginia,MG,EROPS
23897,1222570.0,17000.0,1.0,1001012.0,3414.0,121.0,3.0,1000.0,13507.0,12/22/11 0:00,Virginia,TEX,EROPS
23898,1222571.0,16000.0,1.0,1023061.0,1528.0,121.0,3.0,1000.0,5417.0,12/22/11 0:00,Virginia,TTT,OROPS
23899,1222575.0,48000.0,0.0,1061091.0,28920.0,121.0,3.0,2005.0,0.0,12/22/11 0:00,Virginia,WL,EROPS w AC


In [39]:
train.describe(include='all')

,SalesID,SalePrice,under_20k,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saledate,state,ProductGroup,Enclosure
count,2.390000e+04,23900.000000,23900.000000,2.390000e+04,23900.000000,23900.0,23900.0,23900.000000,23900.000000,23900,23900,23900,23884
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,823,52,6,3
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/29/09 0:00,Texas,TEX,OROPS
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,2823,6579,9399
mean,1.179777e+06,33888.044477,0.375941,8.563724e+05,8326.160795,121.0,3.0,1821.906444,4649.534142,NaN,NaN,NaN,NaN
std,2.392394e+04,24591.558640,0.484375,3.120474e+05,7562.924092,0.0,0.0,384.023768,6066.597357,NaN,NaN,NaN,NaN
min,1.139246e+06,4750.000000,0.000000,3.230000e+02,28.000000,121.0,3.0,1000.000000,0.000000,NaN,NaN,NaN,NaN
25%,1.159416e+06,15000.000000,0.000000,7.718310e+05,3357.000000,121.0,3.0,1996.000000,930.000000,NaN,NaN,NaN,NaN
50%,1.178196e+06,26500.000000,0.000000,1.018198e+06,4666.000000,121.0,3.0,2001.000000,2638.000000,NaN,NaN,NaN,NaN
75%,1.200834e+06,45625.000000,1.000000,1.043985e+06,13395.000000,121.0,3.0,2004.000000,6470.000000,NaN,NaN,NaN,NaN


In [40]:
train['datesale'] = train.apply(lambda row: dateutil.parser.parse(row['saledate']).year, axis=1)

In [41]:
train

,SalesID,SalePrice,under_20k,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saledate,state,ProductGroup,Enclosure,datesale
0,1139246.0,66000.0,0.0,999089.0,3157.0,121.0,3.0,2004.0,68.0,11/16/06 0:00,Alabama,WL,EROPS w AC,2006
1,1139248.0,57000.0,0.0,117657.0,77.0,121.0,3.0,1996.0,4640.0,3/26/04 0:00,North Carolina,WL,EROPS w AC,2004
2,1139249.0,10000.0,1.0,434808.0,7009.0,121.0,3.0,2001.0,2838.0,2/26/04 0:00,New York,SSL,OROPS,2004
3,1139251.0,38500.0,0.0,1026470.0,332.0,121.0,3.0,2001.0,3486.0,5/19/11 0:00,Texas,TEX,EROPS w AC,2011
4,1139253.0,11000.0,1.0,1057373.0,17311.0,121.0,3.0,2007.0,722.0,7/23/09 0:00,New York,SSL,EROPS,2009
5,1139255.0,26500.0,0.0,1001274.0,4605.0,121.0,3.0,2004.0,508.0,12/18/08 0:00,Arizona,BL,OROPS,2008
6,1139256.0,21000.0,0.0,772701.0,1937.0,121.0,3.0,1993.0,11540.0,8/26/04 0:00,Florida,TEX,EROPS,2004
7,1139261.0,27000.0,0.0,902002.0,3539.0,121.0,3.0,2001.0,4883.0,11/17/05 0:00,Illinois,BL,OROPS,2005
8,1139272.0,21500.0,0.0,1036251.0,36003.0,121.0,3.0,2008.0,302.0,8/27/09 0:00,Texas,TEX,EROPS,2009
9,1139275.0,65000.0,0.0,1016474.0,3883.0,121.0,3.0,1000.0,20700.0,8/9/07 0:00,Florida,WL,EROPS w AC,2007


In [42]:
train['yearsold'] = train.datesale - train.YearMade

In [43]:
train.head()

,SalesID,SalePrice,under_20k,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saledate,state,ProductGroup,Enclosure,datesale,yearsold
0,1139246.0,66000.0,0.0,999089.0,3157.0,121.0,3.0,2004.0,68.0,11/16/06 0:00,Alabama,WL,EROPS w AC,2006,2.0
1,1139248.0,57000.0,0.0,117657.0,77.0,121.0,3.0,1996.0,4640.0,3/26/04 0:00,North Carolina,WL,EROPS w AC,2004,8.0
2,1139249.0,10000.0,1.0,434808.0,7009.0,121.0,3.0,2001.0,2838.0,2/26/04 0:00,New York,SSL,OROPS,2004,3.0
3,1139251.0,38500.0,0.0,1026470.0,332.0,121.0,3.0,2001.0,3486.0,5/19/11 0:00,Texas,TEX,EROPS w AC,2011,10.0
4,1139253.0,11000.0,1.0,1057373.0,17311.0,121.0,3.0,2007.0,722.0,7/23/09 0:00,New York,SSL,EROPS,2009,2.0


In [44]:
train2 = train.drop(['saledate', "auctioneerID"], axis=1)

In [45]:
train2.head()

,SalesID,SalePrice,under_20k,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,state,ProductGroup,Enclosure,datesale,yearsold
0,1139246.0,66000.0,0.0,999089.0,3157.0,121.0,2004.0,68.0,Alabama,WL,EROPS w AC,2006,2.0
1,1139248.0,57000.0,0.0,117657.0,77.0,121.0,1996.0,4640.0,North Carolina,WL,EROPS w AC,2004,8.0
2,1139249.0,10000.0,1.0,434808.0,7009.0,121.0,2001.0,2838.0,New York,SSL,OROPS,2004,3.0
3,1139251.0,38500.0,0.0,1026470.0,332.0,121.0,2001.0,3486.0,Texas,TEX,EROPS w AC,2011,10.0
4,1139253.0,11000.0,1.0,1057373.0,17311.0,121.0,2007.0,722.0,New York,SSL,EROPS,2009,2.0


In [46]:
def proc_col(col, train_col=None):
    """Encodes a pandas column with continous ids. 
    """
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

In [47]:

def encode_data(df, train=None):
    """ Encodes rating data with continous user and movie ids. 
    If train is provided, encodes df with the same encoding as train.
    """
    df = df.copy()
    for col_name in ["state", "ProductGroup", "Enclosure"]:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        _,col,_ = proc_col(df[col_name], train_col)
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

In [48]:
data = encode_data(train2)

In [49]:
data.head()

,SalesID,SalePrice,under_20k,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,state,ProductGroup,Enclosure,datesale,yearsold
0,1139246.0,66000.0,0.0,999089.0,3157.0,121.0,2004.0,68.0,0,0,0,2006,2.0
1,1139248.0,57000.0,0.0,117657.0,77.0,121.0,1996.0,4640.0,1,0,0,2004,8.0
2,1139249.0,10000.0,1.0,434808.0,7009.0,121.0,2001.0,2838.0,2,1,1,2004,3.0
3,1139251.0,38500.0,0.0,1026470.0,332.0,121.0,2001.0,3486.0,3,2,0,2011,10.0
4,1139253.0,11000.0,1.0,1057373.0,17311.0,121.0,2007.0,722.0,2,1,2,2009,2.0


In [50]:
data.describe(include='all')

,SalesID,SalePrice,under_20k,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,state,ProductGroup,Enclosure,datesale,yearsold
count,2.390000e+04,23900.000000,23900.000000,2.390000e+04,23900.000000,23900.0,23900.000000,23900.000000,23900.000000,23900.000000,23900.000000,23900.000000,23900.000000
mean,1.179777e+06,33888.044477,0.375941,8.563724e+05,8326.160795,121.0,1821.906444,4649.534142,16.290084,2.184100,0.858536,2008.424477,186.518033
std,2.392394e+04,24591.558640,0.484375,3.120474e+05,7562.924092,0.0,384.023768,6066.597357,13.107064,1.487749,0.767305,2.121588,384.468595
min,1.139246e+06,4750.000000,0.000000,3.230000e+02,28.000000,121.0,1000.000000,0.000000,0.000000,0.000000,0.000000,2004.000000,0.000000
25%,1.159416e+06,15000.000000,0.000000,7.718310e+05,3357.000000,121.0,1996.000000,930.000000,5.000000,1.000000,0.000000,2007.000000,4.000000
50%,1.178196e+06,26500.000000,0.000000,1.018198e+06,4666.000000,121.0,2001.000000,2638.000000,16.000000,2.000000,1.000000,2009.000000,7.000000
75%,1.200834e+06,45625.000000,1.000000,1.043985e+06,13395.000000,121.0,2004.000000,6470.000000,26.000000,3.000000,1.000000,2010.000000,13.000000
max,1.222575e+06,142000.000000,1.000000,1.069977e+06,37198.000000,121.0,2010.000000,220893.000000,51.000000,5.000000,3.000000,2011.000000,1011.000000


# Regression

In [56]:
y = data.SalePrice

In [57]:
x = data.drop(['SalePrice', "under_20k"], axis=1)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [59]:
len(X_train)

19120

In [60]:
x

,SalesID,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,state,ProductGroup,Enclosure,datesale,yearsold
0,1139246.0,999089.0,3157.0,121.0,2004.0,68.0,0,0,0,2006,2.0
1,1139248.0,117657.0,77.0,121.0,1996.0,4640.0,1,0,0,2004,8.0
2,1139249.0,434808.0,7009.0,121.0,2001.0,2838.0,2,1,1,2004,3.0
3,1139251.0,1026470.0,332.0,121.0,2001.0,3486.0,3,2,0,2011,10.0
4,1139253.0,1057373.0,17311.0,121.0,2007.0,722.0,2,1,2,2009,2.0
5,1139255.0,1001274.0,4605.0,121.0,2004.0,508.0,4,3,1,2008,4.0
6,1139256.0,772701.0,1937.0,121.0,1993.0,11540.0,5,2,2,2004,11.0
7,1139261.0,902002.0,3539.0,121.0,2001.0,4883.0,6,3,1,2005,4.0
8,1139272.0,1036251.0,36003.0,121.0,2008.0,302.0,3,2,2,2009,1.0
9,1139275.0,1016474.0,3883.0,121.0,1000.0,20700.0,5,0,0,2007,1007.0


## linear

In [63]:
regr = linear_model.LinearRegression()

In [64]:
# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [ 1.95134325e-03 -1.08772671e-02  1.28392054e-01  9.16600129e-13
 -1.61230572e+02  8.00475075e-01 -6.07592080e+01  3.27000956e+03
 -1.36857816e+04 -3.25624571e+02 -1.64394000e+02]
Mean squared error: 418257601.00
Variance score: 0.30


In [65]:
y_pred

array([15308.76228324, 20361.80805096, 21858.59306422, ...,
       33147.60545152, 53148.95056587, 32171.53320709])

## Ridge

In [71]:
for a in [0, 0.2, 0.4, 0.6, 0.8, 1]:

    regr = Ridge(alpha=a)

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    print('alpha = ', str(a))
    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))
    print('-------------------------')

alpha =  0
Coefficients: 
 [ 6.93354496e-02 -8.75989383e-03 -1.01218031e+00 -4.81631818e+16
 -3.46264392e+14 -2.03285641e+00 -6.11630211e+01  3.26988809e+03
 -1.36891390e+04  3.46264392e+14 -3.46264392e+14]
Mean squared error: 763940040.24
Variance score: -0.28
-------------------------
alpha =  0.2
Coefficients: 
 [ 1.95109156e-03 -1.08773243e-02  1.28392382e-01  0.00000000e+00
 -1.61225312e+02  8.00482577e-01 -6.07593820e+01  3.26999244e+03
 -1.36855062e+04 -3.25611526e+02 -1.64388872e+02]
Mean squared error: 418257809.51
Variance score: 0.30
-------------------------
alpha =  0.4
Coefficients: 
 [ 1.95083988e-03 -1.08773815e-02  1.28392709e-01  0.00000000e+00
 -1.61218254e+02  8.00490078e-01 -6.07595560e+01  3.26997531e+03
 -1.36852309e+04 -3.25600280e+02 -1.64381944e+02]
Mean squared error: 418258018.10
Variance score: 0.30
-------------------------
alpha =  0.6
Coefficients: 
 [ 1.95058822e-03 -1.08774387e-02  1.28393036e-01  0.00000000e+00
 -1.61212335e+02  8.00497578e-01 -6.0759

/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.061562e-16
  overwrite_a=True).T


In [90]:
for a in [0.18, 0.19, 0.2, 0.21, 0.22, 0.23]:

    regr = Ridge(alpha=a)

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    print('alpha = ', str(a))
    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))
    print('-------------------------')

alpha =  0.18
Coefficients: 
 [ 1.95111673e-03 -1.08773186e-02  1.28392349e-01  0.00000000e+00
 -1.61225920e+02  8.00481827e-01 -6.07593646e+01  3.26999415e+03
 -1.36855337e+04 -3.25612749e+02 -1.64389466e+02]
Mean squared error: 418257788.66
Variance score: 0.30
-------------------------
alpha =  0.19
Coefficients: 
 [ 1.95110414e-03 -1.08773214e-02  1.28392365e-01  0.00000000e+00
 -1.61225339e+02  8.00482202e-01 -6.07593733e+01  3.26999329e+03
 -1.36855200e+04 -3.25612414e+02 -1.64388892e+02]
Mean squared error: 418257799.08
Variance score: 0.30
-------------------------
alpha =  0.2
Coefficients: 
 [ 1.95109156e-03 -1.08773243e-02  1.28392382e-01  0.00000000e+00
 -1.61225312e+02  8.00482577e-01 -6.07593820e+01  3.26999244e+03
 -1.36855062e+04 -3.25611526e+02 -1.64388872e+02]
Mean squared error: 418257809.51
Variance score: 0.30
-------------------------
alpha =  0.21
Coefficients: 
 [ 1.95107898e-03 -1.08773271e-02  1.28392398e-01  0.00000000e+00
 -1.61224751e+02  8.00482952e-01 -6.

/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.554057e-17
  overwrite_a=True).T
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.008485e-16
  overwrite_a=True).T
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.061562e-16
  overwrite_a=True).T


In [92]:
for a in [0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.7]:

    regr = Ridge(alpha=a)

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    print('alpha = ', str(a))
    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))
    print('-------------------------')

alpha =  0.1
Coefficients: 
 [ 1.95121740e-03 -1.08772957e-02  1.28392218e-01  0.00000000e+00
 -1.61228856e+02  8.00478826e-01 -6.07592950e+01  3.27000100e+03
 -1.36856439e+04 -3.25617135e+02 -1.64392349e+02]
Mean squared error: 418257705.25
Variance score: 0.30
-------------------------
alpha =  0.11
Coefficients: 
 [ 1.95120482e-03 -1.08772985e-02  1.28392234e-01  0.00000000e+00
 -1.61228053e+02  8.00479201e-01 -6.07593037e+01  3.27000014e+03
 -1.36856301e+04 -3.25617023e+02 -1.64391553e+02]
Mean squared error: 418257715.67
Variance score: 0.30
-------------------------
alpha =  0.12
Coefficients: 
 [ 1.95119223e-03 -1.08773014e-02  1.28392251e-01  0.00000000e+00
 -1.61227695e+02  8.00479576e-01 -6.07593124e+01  3.26999929e+03
 -1.36856163e+04 -3.25616465e+02 -1.64391202e+02]
Mean squared error: 418257726.10
Variance score: 0.30
-------------------------
alpha =  0.13
Coefficients: 
 [ 1.95117965e-03 -1.08773043e-02  1.28392267e-01  0.00000000e+00
 -1.61227165e+02  8.00479951e-01 -6.

/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number5.307793e-17
  overwrite_a=True).T
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number5.838591e-17
  overwrite_a=True).T
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number6.369374e-17
  overwrite_a=True).T
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to

## Lasso

In [72]:
for a in [0, 0.2, 0.4, 0.6, 0.8, 1]:

    regr = linear_model.Lasso(alpha=a)

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    print('alpha = ', str(a))
    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))
    print('-------------------------')

/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


alpha =  0
Coefficients: 
 [ 1.95134325e-03 -1.08772671e-02  1.28392054e-01  0.00000000e+00
  5.27928021e+00  8.00475075e-01 -6.07592080e+01  3.27000956e+03
 -1.36857816e+04 -4.92134423e+02  2.11585219e+00]
Mean squared error: 418257601.00
Variance score: 0.30
-------------------------
alpha =  0.2
Coefficients: 
 [ 1.94732410e-03 -1.08774212e-02  1.28389354e-01  0.00000000e+00
  5.27514744e+00  8.00482359e-01 -6.07582856e+01  3.26991438e+03
 -1.36853732e+04 -4.92030133e+02  2.11148863e+00]
Mean squared error: 418257744.51
Variance score: 0.30
-------------------------
alpha =  0.4
Coefficients: 
 [ 1.94330495e-03 -1.08775753e-02  1.28386655e-01  0.00000000e+00
  5.27101468e+00  8.00489643e-01 -6.07573633e+01  3.26981920e+03
 -1.36849648e+04 -4.91925842e+02  2.10712506e+00]
Mean squared error: 418257888.28
Variance score: 0.30
-------------------------
alpha =  0.6
Coefficients: 
 [ 1.93928581e-03 -1.08777293e-02  1.28383955e-01  0.00000000e+00
  5.26688192e+00  8.00496926e-01 -6.07564

## Elastic Net

In [73]:
for a in [0, 0.2, 0.4, 0.6, 0.8, 1]:

    regr = linear_model.ElasticNet(alpha=a)

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    print('alpha = ', str(a))
    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))
    print('-------------------------')

/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


alpha =  0
Coefficients: 
 [ 1.95134325e-03 -1.08772671e-02  1.28392054e-01  0.00000000e+00
  5.27928021e+00  8.00475075e-01 -6.07592080e+01  3.27000956e+03
 -1.36857816e+04 -4.92134423e+02  2.11585219e+00]
Mean squared error: 418257601.00
Variance score: 0.30
-------------------------
alpha =  0.2
Coefficients: 
 [-1.81447077e-04 -1.13441265e-02  1.30691043e-01  0.00000000e+00
  4.96660340e+00  8.60001910e-01 -6.21619769e+01  3.11522203e+03
 -1.14770290e+04 -3.42362096e+02  7.54373158e-01]
Mean squared error: 422599616.41
Variance score: 0.29
-------------------------
alpha =  0.4
Coefficients: 
 [-1.47706023e-03 -1.16885398e-02  1.32082847e-01  0.00000000e+00
  4.82008485e+00  9.02091622e-01 -6.31952279e+01  2.97598486e+03
 -9.88272856e+03 -2.38534440e+02 -1.35216045e-01]
Mean squared error: 429090896.23
Variance score: 0.28
-------------------------
alpha =  0.6
Coefficients: 
 [-2.26474672e-03 -1.19545218e-02  1.32930982e-01  0.00000000e+00
  4.77287391e+00  9.33229136e-01 -6.39909

In [74]:
for a in [0, 0.2, 0.4, 0.6, 0.8, 1]:

    regr = linear_model.ElasticNet(alpha=0, l1_ratio=a)

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    print('alpha = ', str(a))
    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))
    print('-------------------------')

/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


alpha =  0
Coefficients: 
 [ 1.95134325e-03 -1.08772671e-02  1.28392054e-01  0.00000000e+00
  5.27928021e+00  8.00475075e-01 -6.07592080e+01  3.27000956e+03
 -1.36857816e+04 -4.92134423e+02  2.11585219e+00]
Mean squared error: 418257601.00
Variance score: 0.30
-------------------------
alpha =  0.2
Coefficients: 
 [ 1.95134325e-03 -1.08772671e-02  1.28392054e-01  0.00000000e+00
  5.27928021e+00  8.00475075e-01 -6.07592080e+01  3.27000956e+03
 -1.36857816e+04 -4.92134423e+02  2.11585219e+00]
Mean squared error: 418257601.00
Variance score: 0.30
-------------------------
alpha =  0.4
Coefficients: 
 [ 1.95134325e-03 -1.08772671e-02  1.28392054e-01  0.00000000e+00
  5.27928021e+00  8.00475075e-01 -6.07592080e+01  3.27000956e+03
 -1.36857816e+04 -4.92134423e+02  2.11585219e+00]
Mean squared error: 418257601.00
Variance score: 0.30
-------------------------
alpha =  0.6
Coefficients: 
 [ 1.95134325e-03 -1.08772671e-02  1.28392054e-01  0.00000000e+00
  5.27928021e+00  8.00475075e-01 -6.07592

In [76]:
for a in [3, 10, 18, 50, 100, 1000]:

    regr = linear_model.ElasticNet(alpha=0, max_iter=a)

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    print('alpha = ', str(a))
    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))
    print('-------------------------')

/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/Users/chrispaul/anaconda2/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


alpha =  3
Coefficients: 
 [-9.31445297e-04 -1.09747225e-02  1.31362963e-01  0.00000000e+00
  5.23783572e+00  8.07259837e-01 -6.06477363e+01  3.27237151e+03
 -1.36469352e+04 -4.65112075e+02  2.04766015e+00]
Mean squared error: 418427184.07
Variance score: 0.30
-------------------------
alpha =  10
Coefficients: 
 [ 1.94288381e-03 -1.08773894e-02  1.28388620e-01  0.00000000e+00
  5.27922274e+00  8.00477128e-01 -6.07590793e+01  3.27000972e+03
 -1.36857166e+04 -4.92061726e+02  2.11573827e+00]
Mean squared error: 418257650.54
Variance score: 0.30
-------------------------
alpha =  18
Coefficients: 
 [ 1.95133561e-03 -1.08772672e-02  1.28392051e-01  0.00000000e+00
  5.27928015e+00  8.00475077e-01 -6.07592078e+01  3.27000956e+03
 -1.36857815e+04 -4.92134357e+02  2.11585209e+00]
Mean squared error: 418257601.04
Variance score: 0.30
-------------------------
alpha =  50
Coefficients: 
 [ 1.95134325e-03 -1.08772671e-02  1.28392054e-01  0.00000000e+00
  5.27928021e+00  8.00475075e-01 -6.07592080

## RF

In [81]:
parameters = []
mse = []

for n in [2, 5, 10, 20, 50, 100]:
    for md in [3,5,10,20,50,100,1000]:
        for mss in [2, 3, 5, 8, 10, 20]:

            regr = RandomForestRegressor(max_depth=md, n_estimators=n, min_samples_split=mss)
            
            regr.fit(X_train, y_train)
            
            y_pred = regr.predict(X_test)
            
            mse_ = np.sqrt(np.mean((y_pred - y_test)**2))
            
            parameters.append("n_estimators = " + str(n)
                              + ", max_depth = " + str(md)
                              + ", min_samples_split = " + str(mss))
            mse.append(mse_)
            
    print("a sixth")

a sixth
a sixth
a sixth
a sixth
a sixth
a sixth


In [85]:
ind = np.argmin(mse)

In [86]:
ind

236

In [87]:
mse[236]

9758.55780823173

In [89]:
parameters[236]

'n_estimators = 100, max_depth = 50, min_samples_split = 5'